In [2]:
#Importaciones de las bibliotecas
from IPython.display import clear_output as clear
from ipywidgets import interactive
import ipywidgets as widgets
import random
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy
import math
import pylab 
import matplotlib.image as img
from matplotlib.offsetbox import  AnnotationBbox, OffsetImage

#-Función principal para la formación de las matrices

def mov_particula (posx, posy, mat,n,tamano_subcelda):
    #El bucle examina la matriz de izquierda a derecha y de arriba hacia abajo.
    while posy < n:        
        while posx < n:            
            if mat[posy][posx] == 1:
                new_mat = donde_mueve (posx, posy, mat,n,tamano_subcelda)
                posx = posx + 1                    
            #Si en la posición dada hay un cero, el bucle pasa a la siquiente posición.
            else:            
                posx = posx + 1            
        #Una vez examinadas todas las casillas de una fila, se examina la siguiente fila comenzando desde la columna 0.      
        else:               
            posx = 0
            posy = posy + 1      
    #Una vez completada la iteración se iguala la matriz original a la editable y se repite el proceso hasta completar las iteraciones deseadas.        
    else: 
        posx = 0
        posy = 0 
        mat = new_mat
        return (mat)

#Función para determinar el movimiento de la partícula    

def donde_mueve (posx, posy, mat,n,tamano_subcelda):
    xy = random.randint(1, 2) 
    #Caso para cuando el movimiento es horizontal.
    if xy == 1: 
        #Si la partícula se encuentra en el extremo izquierdo, esta solo puede moverse a la derecha  
        if posx == 0:
            #La variable r determina si la partícula se mueve en cierta dirección o si se queda quieta
            r = random.randint(0, 1)       
            if al_lado(posx + r, posy, mat) == True:
                mat[posy][posx] = 1
                return (mat)
            else:
                mat[posy][posx] = 0
                mat[posy][posx + r] = 1
                return (mat)
        #Si la partícula se encuentra en el extremo derecho, esta solo puede moverse a la izquierda        
        elif posx == n - 1: 
            r = random.randint(-1, 0)
            if al_lado(posx + r, posy, mat) == True:
                mat[posy][posx] = 1
                return (mat)
            else:
                mat[posy][posx] = 0
                mat[posy][posx + r] = 1
                return (mat)
        else:
            r = random.randint(-1, 1)
            if al_lado(posx + r, posy, mat) == True:
                mat[posy][posx] = 1
                return (mat)
            else:
                mat[posy][posx] = 0
                mat[posy][posx + r] = 1
                return (mat)
    #Caso para cuando el movimiento es vertical
    else: 
        #Si la partícula se encuentra en el extremo superior, esta solo puede moverse hacia abajo          
        if posy == 0:       
            r = random.randint(0, 1)
            if al_lado(posx, posy + r, mat) == True:
                mat[posy][posx] = 1
                return (mat)
            else:
                mat[posy][posx] = 0
                mat[posy + r][posx] = 1
                return (mat)
        #Si la partícula se encuentra en el extremo inferior, esta solo puede moverse hacia arriba
        elif posy == n - 1:     
            r = random.randint(-1, 0)
            if al_lado(posx, posy + r, mat) == True:
                mat[posy][posx] = 1
                return (mat)
            else:        
                mat[posy][posx] = 0                    
                mat[posy + r][posx] = 1
                return (mat)
        else:
            r = random.randint(-1, 1)
            if al_lado(posx, posy + r, mat) == True:
                mat[posy][posx] = 1
                return (mat)
            else:
                mat[posy][posx] = 0
                mat[posy + r][posx] = 1
                return (mat)


#Función que verifica si en la posición dada se encuentra ya una partícula o no en la matriz

def al_lado (posx, posy, mat):
    if mat[posy][posx] == 1:
        return True
    else:
        return False


#Función que determina cuantas partículas hay en una celda

def cafe_leche (posx, posy, mat, lista_particulas, lista_entropia,n,tamano_subcelda, tiempo):

    #Se utiliza la función para generar matrices, elaborada con anterioridad.
    if tiempo == 0:
        return mat, lista_entropia
        
    mat = mov_particula (0, 0, mat,n,tamano_subcelda)
    while posy < n:
        while posx < n:
            #Se generan celdas, donde cada una es un estado para la partícula.
            celda = mat[posy : posy + tamano_subcelda, posx : posx + tamano_subcelda]
            posx = posx + tamano_subcelda
            #Se crea una lista con la cantidad de partículas que hay por cada celda.
            lista_particulas = lista_particulas + [cant_part (celda, 0, 0, tamano_subcelda)]
        else:
            posx = 0
            posy = posy + tamano_subcelda
    else:
        #Una vez analizada toda la matriz se crea una lista con los valores de entropía para cada matriz.
        lista_entropia.append(calc_entropia (lista_particulas, 0, 0, 0,n,tamano_subcelda))
        
        return mat, lista_entropia     

        
#Función que determina cuantas partículas hay en una celda

def calc_entropia (lista_particulas, cant_celdas, prob, entropia,n,tamano_subcelda):
    #Toma la catidad de partículas por cada celda y calcula su probabilidad, 
    #la cual es la cantidad de partículas de la celda divido entre el total de partícula en la matriz.
    while cant_celdas < (n**2/tamano_subcelda**2): 
        prob = lista_particulas [cant_celdas]/(tamano_subcelda**2)
        if prob == 0:
            cant_celdas = cant_celdas + 1
        #Se calcula la entropía con la formulación de Gibbs.
        else:
            logaritmo = math.log(prob)
            entropia = entropia - prob * logaritmo
            prob = 0
            cant_celdas = cant_celdas + 1
    else:
        return entropia
        

#Función que determina cuantas partículas hay en una celda

def cant_part (celda, x_celda, y_celda, tamano_subcelda):
    #Toma una celda dada y cuenta cuantas partículas hay en ella
    part_celda = 0
    while y_celda < tamano_subcelda:
        while x_celda < tamano_subcelda:
            if celda [y_celda][x_celda] == 0:
                x_celda = x_celda + 1
            else:
                x_celda = x_celda + 1
                part_celda = part_celda + 1
        else:
            y_celda = y_celda + 1
            x_celda = 0
    else:
        y_celda = 0
        x_celda = 0
        return part_celda


#Ejecución del programa

def Entropia():
    n = 40
    tamano_subcelda = 10

    lista_iteraciones = []
    lista_entropia = [] 

    mat = numpy.arange(n**2).reshape(n, n)
    mat[0:, ::1] = 0
    mat[(n//2)-tamano_subcelda//2:(n//2)+tamano_subcelda//2, (n//2)-tamano_subcelda//2:(n//2)+tamano_subcelda//2] = 1

    print("Se recomienda limpiar las gráficas antes de generar una nueva simulación")
    
    def GraficoInteractivoEntropia(tiempo):       
        posy = 0
        posx = 0
        lista_particulas = []
        
        iteracion = 1 + tiempo
        
        solución = cafe_leche (posx, posy, mat, lista_particulas, lista_entropia,n,tamano_subcelda, tiempo)
        
        matriz= solución[0]
        listaEntropia = solución [1]
     

        fig, (ax1, ax2) = plt.subplots(1, 2, dpi=120,figsize=(14,7), sharey= False)
        
        im = plt.imread('taza.png')
        newax = fig.add_axes([-0.0415,0.107,0.8995,0.8995], anchor='W', zorder=1)
        newax.imshow(im)
        newax.axis('off')    
        
        #pylab.title('matriz a ' + str(iteracion - 1) + ' pasos') 
        y =  numpy.where (matriz ==1) [0]
        x =  numpy.where (matriz ==1) [1]
        ax1.add_patch(
                    patches.Rectangle((-1, -1), n+1, n+1, color = "#362518", fill=True) )
        ax1.plot (x, y, 'ws')
        ax1.set_xlim (0, n)
        ax1.set_ylim (0, n)
        ax1.axis('off')

        ax2.set_title("Evolucion de entropía", fontsize=16) 
        ax2.set_xlabel ("Pasos", fontsize=14)
        ax2.set_ylabel ("Entropía", fontsize=14) 
        ax2.plot(list(range(0,len(listaEntropia))), listaEntropia)     
        ax2.set_xlim (-10, 300)
        ax2.set_ylim (1.25, 2.75)

        plt.show ()
        
    play = widgets.Play(value=0, min=0, max=1900, step=1, disabled=False)
    tieEtiqueta = widgets.Label (value="Tiempo $(ms)$:")
    tieSlider = widgets.FloatSlider(min=0, max=1900,)
    widgets.jslink((play, 'value'), (tieSlider, 'value'))
    cajaTiempo = widgets.HBox([tieEtiqueta, tieSlider])


    button = widgets.Button(description="Limpiar")
    output = widgets.Output()

    salida = widgets.interactive_output(GraficoInteractivoEntropia, {'tiempo':tieSlider,})
    
    def on_button_clicked(b):
            clear()
            Entropia ()

    button.on_click(on_button_clicked)
    
    display(button, play, salida)


## Entropía

La entropía se puede definir como una medida cuantitativa para el grado del desorden o aleatoriedad en un sistema. Un ejemplo de esto es al lanzar un vaso de vidrio al suelo, en donde se espera que al chocar con el suelo este se rompa en varios trozos de vidrio. Por otra parte, no se esperaría nunca que al lanzar un grupo de trozos de vidrio se construya un vaso completo. Por lo que, en un sistema la entropía siempre se espera que aumente o se mantenga, más nunca disminuye. 

Otro ejemplo es el mostrado en la siguiente simulación, en donde se coloca un poco de crema en el centro de una taza de café. Se espera que la crema poco a poco se vaya esparciendo por el café, pero no que está se reagrupre nuevamente en el centro de la taza.

Dicho fenómeno se expresa por la ecuación:

$$
S=k\cdot ln\Omega
$$

Donde:

$S$: Entropía en el sistema.\
$k$: Constante de Boltzmann.\
$\Omega$: Microestados posibles para el sistema. Para este caso, son las posiciones posibles que pueden tener cada partícula de crema en el café.

In [3]:
Entropia ()

Se recomienda limpiar las gráficas antes de generar una nueva simulación


Button(description='Limpiar', style=ButtonStyle())

Play(value=0, max=1900)

Output()